# Code for Estimating the Ground State Energy of the Hydroxyl Cation using BQSKit

This Jupyter notebook walks through the workflow of circuit synthesis and evaluation to estimate the ground state energy of the hydyoxyl radical. Note that this script will save a QASM string in your current working directory due to how the compilation process is implemented.

This notebook is heavily inspired by the provided [example code](https://github.com/qccontest/QC-Contest-Demo/blob/main/examplecode.ipynb) and [noise model code](https://github.com/qccontest/QC-Contest-Demo/blob/main/NoiseModel_and_SystemModel.ipynb).

## Basic Installation
Install required packages.

In [ ]:
!pip install qiskit
!pip install qiskit-nature[pyscf] -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.7/112.7 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 MB 33.3 MB/s eta 0:00:00


In [ ]:
!pip install qiskit_aer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 89.4 MB/s eta 0:00:00


In [ ]:
!pip install bqskit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.3/396.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 17.2 MB/s eta 0:00:00


In [ ]:
# Used for modeling the hydroxyl radical
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper,ParityMapper,QubitConverter

# Used for VQE
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.algorithms.optimizers import SLSQP
from qiskit_aer.primitives import Estimator
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

# Used for transpilation and circuit modeling in Qiskit
import numpy as np
import pylab
import qiskit.providers
from qiskit import Aer,pulse, QuantumCircuit, transpile
from qiskit.utils import QuantumInstance, algorithm_globals
import time

# Used for circuit synthesis and optimization
from bqskit import compile, Circuit

# Used for converting the provided Hamiltonian file into
# a usable Pauli representation
from qiskit.quantum_info import SparsePauliOp

# Used for noise modeling
from qiskit.providers.aer.noise import NoiseModel
import qiskit.providers.aer.noise as noise

# Used for quantum hardware backend
from qiskit.providers.fake_provider import *

# Used for reading noise model files
import pickle

## Initialize Configuration Variables
Here, the user can set the seed to use, the number of shots used throughout the workflow, and the paths to the noise model and Hamiltonian.

For reproducibility, the user should provide a path to one of the [provided noise models](https://github.com/qccontest/QC-Contest-Demo/tree/main/NoiseModel) and to the [provided Hamiltonian file](https://github.com/qccontest/QC-Contest-Demo/blob/main/Hamiltonian/OHhamiltonian.txt).

Note that the noise model name is purely used for the filename which stores an intermediate QASM string in the current working directory as part of the compilation process.

In [ ]:
# Set the seed used throughout the workflow.
seed = 20
algorithm_globals.random_seed = seed
seed_transpiler = seed

# Set the number of shots used throughout the workflow.
shots = 2852

# Sets the noise model name, which is used for saving a QASM file
# in the current working directory.
noisemodel_name = 'fakekolkata'

# Set the noise model used in the workflow.
noisemodel_path = './NoiseModel/' + noisemodel_name + '.pkl'


# Sets the hamiltonian used in the workflow.
hamiltonian_file_path = './Hamiltonian/OHhamiltonian.txt'

## Generate Hamiltonian and Pauli String

The code sets up PySCF to generate the hamiltonian of the hydroxyl cation with the basis function as 'sto3g' to fit the spin orbital, and then uses the JordanWignerMapper to map the Fermionic terms to Pauli strings.

Note: this code reads the Hamiltonian from the 'hamiltonian_file_path' when it needs to evaluate the correctness of the circuit.

In [ ]:
# Define a string representing a simplified molecular structure of Alanine,
# specifying the coordinates of Oxygen (O) and Hydrogen (H) atoms.
ultra_simplified_ala_string = """
O 0.0 0.0 0.0
H 0.45 -0.1525 -0.8454
"""

# Create a PySCF Driver for the hydroxyl cation using the 'sto3g' basis
# function to fit the spin orbital.
driver = PySCFDriver(
    atom=ultra_simplified_ala_string.strip(),
    basis='sto3g',
    charge=1,
    spin=0,
    unit=DistanceUnit.ANGSTROM
)

# Obtain a qmolecule containing molecular information generated by the driver.
qmolecule = driver.run()

/usr/local/lib/python3.10/dist-packages/pyscf/dft/libxc.py:772: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, the same to the B3LYP functional in Gaussian and ORCA (issue 1480). To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


In [ ]:
# Obtain the Hamiltonian.
hamiltonian = qmolecule.hamiltonian
coefficients = hamiltonian.electronic_integrals
# print(coefficients.alpha)

# Obtain the second quantized operations from the Hamiltonian.
second_q_op = hamiltonian.second_q_op()

Polynomial Tensor
 "+-":
[[-3.21461222e+01  5.59899100e-01  1.87617178e-01  3.74538735e-16
   6.19498212e-16 -1.94702445e-01]
 [ 5.59899100e-01 -7.35898345e+00 -2.46352634e-01 -9.87250577e-16
  -1.62475968e-15  9.51226718e-01]
 [ 1.87617178e-01 -2.46352634e-01 -6.56995119e+00  3.42558853e-15
   3.68881527e-15 -1.09726793e+00]
 [ 3.75150523e-16 -1.00302259e-15  3.31209337e-15 -6.94886145e+00
  -5.86603488e-16 -1.24551876e-15]
 [ 6.18412765e-16 -1.57506805e-15  4.06262679e-15 -4.56444076e-16
  -6.94886145e+00 -3.85804477e-15]
 [-1.94702445e-01  9.51226718e-01 -1.09726793e+00 -1.14788360e-15
  -4.58668558e-15 -4.64967973e+00]]
 "++--":
[[[[ 4.74977044e+00 -4.38465691e-01 -1.51436760e-01 -3.01805209e-16
    -4.70660215e-16  1.59790984e-01]
   [-4.38465691e-01  6.47204045e-02  1.84429506e-02  3.96633254e-17
     7.05655955e-17 -2.66865302e-02]
   [-1.51436760e-01  1.84429506e-02  2.46189939e-02 -1.27230283e-17
    -2.00835315e-17  6.40512562e-03]
   [-3.01807372e-16  3.98690260e-17 -1.30785

In [ ]:
# Create an instance of the JordanWignerMapper.
mapper = JordanWignerMapper()

# Create a converter from fermionic operators to qubit operators,
# using the Jordan Wigner Mapper. Do not reduce the number
# of required qubits by two.
converter = QubitConverter(mapper=mapper, two_qubit_reduction=False)

# Convert the second quantized operator (second_q_op) to a qubit operator (qubit_op).
qubit_op = converter.convert(second_q_op)

<ipython-input-12-60b4e97714fb>:2: DeprecationWarning: The QubitConverter class is deprecated as of version 0.6.0 and will be removed no sooner than 3 months after the release . Instead you should directly use the QubitMapper instance which you used to pass into the QubitConverter as the first argument. Refer to the documentation of the qiskit_nature.second_q.mappers module for more information.
  converter = QubitConverter(mapper=mapper, two_qubit_reduction=False)
/usr/local/lib/python3.10/dist-packages/qiskit_nature/deprecation.py:297: PauliSumOpDeprecationWarning: PauliSumOp is deprecated as of version 0.6.0 and support for them will be removed no sooner than 3 months after the release. Instead, use SparsePauliOp. You can switch to SparsePauliOp immediately, by setting `qiskit_nature.settings.use_pauli_sum_op` to `False`.
  return func(*args, **kwargs)


## Obtain Reference Energy using Classical Simulation
We use the classical minimum eigensolver to obtain a reference energy of the hydroxyl radical.

In [ ]:
# Create a solver for the ground state energy, which applies the
# Jordan-Wigner transformation and uses a classical eigensolver.
solver = GroundStateEigensolver(
    JordanWignerMapper(),
    NumPyMinimumEigensolver(),
)

In [ ]:
# Obtain the reference energy.
result = solver.solve(qmolecule)
print(result.computed_energies)

[-78.75252123]


In [ ]:
# Print the nuclear repulsion energy.
print(result.nuclear_repulsion_energy)

4.36537496654537


In [ ]:
# Obtain the ground state energy, which is the sum of the computed
# energies and the nuclear repulsion energy.
ref_value = result.computed_energies + result.nuclear_repulsion_energy
ref_value_float = ref_value[0]
print(ref_value_float)

[-74.38714627]
-74.38714626818725


In [ ]:
# print(qmolecule.num_spatial_orbitals)
# print(qmolecule.num_particles)
# print(mapper)

6
(4, 4)


## Construct Ansatz using VQE and Circuit Synthesis
At this step, we first perform VQE on the UCCSD ansatz to obtain a parameterized circuit. We then transpile the circuit so it has one and two qubit gates, and proceed to circuit synthesis with BQSKit.


### Obtain parameterized UCCSD Ansatz using VQE
This step of the code obtains the parameterized UCCSD ansatz by running VQE.

In [ ]:
# Create an instance of the UCCSD ansatz using the properties of
# the hydroxyl radical.
ansatz = UCCSD(
    qmolecule.num_spatial_orbitals,
    qmolecule.num_particles,
    mapper,
    initial_state=HartreeFock(
        qmolecule.num_spatial_orbitals,
        qmolecule.num_particles,
        mapper,
    ),
)

In [ ]:
# Create an estimator for the ground state energy of the hydroxyl radical.
# Use the seed and specified number of shots.
estimator = Estimator(
    backend_options = {
        'method': 'statevector',
        'device': 'CPU'
        # 'noise_model': noise_model
    },
    run_options = {
        'shots': shots,
        'seed': seed,
    },
    transpile_options = {
        'seed_transpiler':seed_transpiler
    }
)

In [ ]:
# Create a VQE solver.
vqe_solver = VQE(estimator, ansatz, SLSQP())
# Set an initial point for the optimization.
vqe_solver.initial_point = [0.0] * ansatz.num_parameters

In [ ]:
# Computes the ground state energy using a simulator
# and optimizes the parameters for the UCCSD ansatz
# through VQE.
start_time = time.time()
calc = GroundStateEigensolver(mapper, vqe_solver)
res = calc.solve(qmolecule)
end_time = time.time()
# print(res)

In [ ]:
# Obtain the optimal parameters for the UCCSD ansatz from VQE.
optimal_point = res.raw_result.optimal_point
# print(optimal_point)

In [ ]:
# Obtain the parameterized UCCSD ansatz as a result of VQE.
optimized_ansatz = vqe_solver.ansatz.bind_parameters(optimal_point)
optimized_ansatz_qasm = optimized_ansatz.qasm()

### Transpile the Parameterized UCCSD ansatz and perform circuit synthesis using BQSKit
We then transpile the parameterized UCCSD ansatz on Qiskit's QASM simulator to obtain a circuit with one and two qubit gates, and then perform BQSKit with an optimization level of 3 (for fine-tuning of gate parameters).

In [ ]:
# Obtain a Qiskit representation of the parameterized ansatz.
circuit = QuantumCircuit.from_qasm_str(optimized_ansatz_qasm)

In [ ]:
# Use the QASM simulator backend.
backend = Aer.get_backend('qasm_simulator')

In [ ]:
# Transpile the circuit on the simulator with optimization level 0, as we
# only want to obtain a circuit with one and two qubit gates.
transpiled_circuit = transpile(circuit, backend, optimization_level=0, seed_transpiler=seed)

In [ ]:
# transpiled_circuit.depth()

In [ ]:
# Set a base file name to save an intermediate QASM file.
base_ansatz_filename = 'UCCSD_VQE_ansatz_seed' + str(seed) + '_' + noisemodel_name

In [ ]:
# Write the circuit in QASM form to the file. This is necessary for
# BQSKit to read in the circuit.
with open(base_ansatz_filename + '_transpiled' + '.qasm', 'w') as file:
  file.write(transpiled_circuit.qasm())

In [ ]:
# Obtain a BQSKit representation of the transpiled circuit.
bqskit_rep_circuit = Circuit.from_file(base_ansatz_filename + '_transpiled' + '.qasm')

In [ ]:
# Compile the circuit using BQSKit, with the optimization level as 3.
compiled_circuit = compile(bqskit_rep_circuit, optimization_level=3, seed=seed)

In [ ]:
# print(compiled_circuit.depth)

## Circuit Evaluation
We now evaluate the circuit with the provided Hamiltonian and use Qiskit's Estimator to estimate the ground state energy.

In [ ]:
# Read in the Hamiltonian as a string.
with open(hamiltonian_file_path, 'r') as file:
  hamiltonian_str = file.read()

In [ ]:
# Define a function for parsing the Hamiltonian string into a list of
# Pauli strings.
def parse_hamiltonian(content: str):
    """
    Parses the input Hamiltonian string and returns a list of Pauli strings
    padded for a circuit with 27 qubits with their coefficients.

    Args:
    content (str): The string representation of the Hamiltonian to parse.
    It is assumed that the file contains one Pauli coefficient and string per
    line, and each line is of the format "<coeff> * <pauli_string>".

    Returns:
    A list of tuples representing a list of the Pauli coefficients and strings.
    Each tuple is of the form (<coeff>, <pauli_string>), where <coeff> is a
    floating point number representing the coefficient, and <pauli_string> is
    a string of length 27 for a circuit with 27 physical qubits.
    """

    # Initializing the list to hold the parsed Pauli operators and their coefficients
    pauli_list = []

    # Splitting the content into lines and parsing each line
    for line in content.splitlines():
        # Removing the '+' symbol and splitting each line into coefficient and operator parts
        parts = line.replace('+', '').replace('- ', '-').split(' * ')

        # If the line is correctly formatted, it should have exactly two parts
        if len(parts) == 2:
            try:
                # The first part is the coefficient
                coefficient = float(parts[0])
                # The second part is the operator string
                # Prepend 15 I's to fit to FakeMontreal hardware backend
                operator = "I" * 15 + parts[1]

                # Appending the parsed operator and coefficient to the list
                pauli_list.append((operator, coefficient))
            except ValueError:  # Handle lines that cannot be parsed correctly
                print('Error: cannot parse this Pauli string:', parts)

    return pauli_list

In [ ]:
# Obtain the observable as a Sparse Pauli Operator
observable = SparsePauliOp.from_list(parse_hamiltonian(hamiltonian_str))
print(len(observable))
print(f">>> Observable: {observable.paulis}")

In [ ]:
# Initialize the system model
system_model = FakeMontreal()

In [ ]:
# Read in the noise model
with open(noisemodel_path, 'rb') as file:
    noise_model = pickle.load(file)

In [ ]:
# Initialize a container for the noise model
noise_model1 = noise.NoiseModel()

In [ ]:
# Parameterize the noise model object with the specified noise model
noise_modelreal = noise_model1.from_dict(noise_model)

In [ ]:
# noise_modelreal

In [ ]:
# Obtain a Qiskit representation of the BQSKit compiled circuit
compiled_circ_qiskit = QuantumCircuit.from_qasm_str(compiled_circuit.to('qasm'))

In [ ]:
# compiled_circ_qiskit.draw()

In [ ]:
# Specify a layout for the transpiler onto the system model.
# Map the first 12 logical qubits to the first 12 physical qubits.
layout = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [ ]:
# Transpile the circuit with the above layout on the system model.
transpiled_circuit_montreal = transpile(compiled_circ_qiskit, backend=system_model, seed_transpiler=seed, initial_layout=layout)

In [ ]:
# transpiled_circuit_montreal.draw()

In [ ]:
# Create an estimator for the ground state energy, using the specified shots
# and the noise model.
# Skip transpilation as we've already transpiled our circuit onto the system
# model.
estimator = Estimator(
    backend_options = {
        'method': 'statevector',
        'device': 'CPU',
        'noise_model': noise_modelreal
    },
    run_options = {
        'shots': shots,
        'seed': seed,
    },
    skip_transpilation=True
)

In [ ]:
# Run the estimator to obtain the ground state energy.
job = estimator.run(transpiled_circuit_montreal,observable)
result = job.result()
print(f">>> {result}")

In [ ]:
# Set the computed energy and the nuclear repulsion energy.
res_computeden = result.values[0]
# The nuclear repulsion energy is a constant based on the input
# molecule.
res_nuc_repulen = qmolecule.nuclear_repulsion_energy

# print(res_computeden)
# print(res_nuc_repulen)

In [ ]:
# Compute the accuracy between the observed energy and the reference energy.
result_energy = res_computeden + res_nuc_repulen
accuracy_score = (1 - abs((result_energy - ref_value_float) / ref_value_float)) * 100
print("Accuracy Score: %f%%" % (accuracy_score))

### Obtain the Duration of the Final Circuit
Here, we use the 'pulse' module from Qiskit to obtain the duration of the
quantum circuit in terms of the time resolution of the system model.

In [ ]:
from qiskit import pulse

In [ ]:
# Build a pulse schedule for the quantum circuit on the specified
# system model
with pulse.build(system_model) as my_program1:
  with pulse.transpiler_settings(optimization_level=0):
    pulse.call(transpiled_circuit_montreal)

In [ ]:
# Print the duration of the quantum circuit
my_program1.duration

In [ ]:
# Print the final transpiled circuit.
transpiled_circuit_montreal.qasm()